# Hugging Face 初體驗

[Hugging Face](https://huggingface.co/) 是一個專注於機器學習和自然語言處理的開源平台。它提供了大量的模型、數據集和工具，支持多種任務如文本生成、圖像分類和語音識別。用戶可以輕鬆地搜索、下載和使用這些資源來開發和部署AI應用。這個平台對研究人員和開發者來說非常有用，因為它促進了技術共享和協作。

平台提供各類開源模型及資料集，也針對不同型態的[任務](https://huggingface.co/tasks)進行分類，便於查找使用．

![Hugging Face](https://raw.githubusercontent.com/jonascheng/learning-transformer-with-huggingface/refs/heads/main/assets/images/huggingface.png)

我們來體驗幾個簡單的任務

## [Token Classification](https://huggingface.co/tasks/token-classification)

Token classification 是自然語言處理中的一種任務，旨在對文本中的每個 token（通常是單詞或子詞）進行分類。這種技術可以用來解決多種問題，以下是幾個例子：

* 命名實體識別（NER）：識別文本中的人名、地名、組織名等實體。例如，從句子「John is working in New York」中識別出「John」是人名，「New York」是地名。
* 詞性標註（POS tagging）：標註每個單詞的詞性，如名詞、動詞、形容詞等。例如，從句子「The quick brown fox jumps over the lazy dog」中標註出「quick」是形容詞，「jumps」是動詞。


In [ ]:
# transformers not support NumPy 2.0 yet
!pip install -q numpy~=1.26.4 torch~=2.2.2 transformers~=4.46.2 accelerate~=1.1.1
# for facebook/mask2former-swin-small-coco-instance
!pip install -q scipy~=1.14.1 pillow~=11.0.0

In [ ]:
import torch

# 檢查是否有 GPU 可以使用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("mps" if torch.backends.mps.is_available() else device)

In [ ]:
from transformers import pipeline

# 命名實體識別（NER）, 讓 pipeline 自動選擇模型
ner_classifier = pipeline(
  task="ner",
  device=device,)
ner_classifier("John is working in New York")

從回傳的結構化資料，我們可以看到每個 token 的 `word` 和 `entity`，這裡的 `entity` 是指該 token 的分類結果。

```json
[{'entity': 'I-PER',
  'word': 'John'},
 {'entity': 'I-LOC',
  'word': 'New'},
 {'entity': 'I-LOC',
  'word': 'York',}]
```

In [ ]:
# 詞性標註（POS tagging）, 指定模型
model = "vblagoje/bert-english-uncased-finetuned-pos"
pos_classifier = pipeline(
  task="token-classification",
  model=model,
  device=device,)
pos_classifier("The quick brown fox jumps over the lazy dog")

## [Zero-Shot Classification](https://huggingface.co/tasks/zero-shot-classification)

Zero-Shot Classification 旨在不需要特定訓練數據的情況下，對文本進行分類。這意味著模型可以在沒有見過特定類別的訓練樣本的情況下，直接對新類別進行分類。這種技術依賴於預訓練模型的強大語言理解能力。以下是幾個可以透過 Zero-Shot Classification 解決的例子：

* 情感分析：在沒有特定情感標註數據的情況下，對文本進行正面、負面或中性情感分類。例如，對一條新的社交媒體評論進行情感分類。
* 主題分類：對新聞文章進行主題分類，如政治、經濟、科技等，而不需要針對每個主題進行單獨訓練。例如，將一篇新的新聞文章分類為「科技」。

Zero-Shot Classification 的優勢在於其靈活性和廣泛應用，特別是在缺乏標註數據的情況下。


In [ ]:
from transformers import pipeline

model = "facebook/bart-large-mnli"
pipe = pipeline(
  task="zero-shot-classification",
  model=model,
  device=device,)
pipe("I have a problem with my iphone that needs to be resolved asap!",
    candidate_labels=["urgent", "not urgent", "phone", "tablet", "computer"],
)

從回傳的結構化資料，我們可以知道這是一個關於 `phone` 且 `urgent` 的用戶反饋。

## Image Classification

Transformer 架構不僅適用於自然語言處理，也可以應用於圖像處理。這裡我們使用一個預訓練的圖像分類模型來識別圖像中的物體。

![貓](https://live.staticflickr.com/5670/30942263876_969b93e461_b.jpg)

來源: https://live.staticflickr.com/5670/30942263876_969b93e461_b.jpg

In [ ]:
from transformers import pipeline

model = "google/vit-base-patch16-224"
pipe = pipeline(
  task="image-classification",
  model=model,
  device=device,)
pipe("https://live.staticflickr.com/5670/30942263876_969b93e461_b.jpg")


Transformer 架構還可以應用於圖像分割任務。

In [ ]:
from transformers import ViTForImageClassification
from pprint import pprint

model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')

pprint(model)

## Image Segmentation

圖像分割是一種圖像處理任務，旨在將圖像中的每個像素分配給特定的類別。這種技術可以應用於多種場景，如醫學影像分析、自然災害檢測和智能交通等。

![行人遛狗](https://images.pexels.com/photos/5748630/pexels-photo-5748630.jpeg)

來源: https://images.pexels.com/photos/5748630/pexels-photo-5748630.jpeg

In [ ]:
from transformers import pipeline

model = "facebook/mask2former-swin-small-coco-instance"
pipe = pipeline(
  task="image-segmentation",
  model=model,
  device=device,)
pipe("https://images.pexels.com/photos/5748630/pexels-photo-5748630.jpeg")
